### `Import Library`

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

### `Function`

In [19]:
def pre_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rows, cols = gray.shape[0], gray.shape[1]
    pre_lbp = np.zeros((rows, cols), dtype=np.uint8)  
    return gray, rows, cols, pre_lbp


def calculate_lbp_clockwise(image):
    gray, rows, cols, lbp_clockwise = pre_lbp(image)

    for i in range(1, rows - 1):  # Hindari satu piksel di tepi atas dan bawah
        for j in range(1, cols - 1):  # Hindari satu piksel di tepi kiri dan kanan
            center = gray[i, j]
            code = 0    
            # Tetangga dalam urutan searah jarum jam mulai dari 3 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j + 1], gray[i, j + 1],
                         gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1], gray[i - 1, j - 1]]
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            lbp_clockwise[i - 1, j - 1] = code
    return lbp_clockwise


def calculate_lbp_anticlockwise(image):
    gray, rows, cols, lbp_anticlockwise = pre_lbp(image)

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0            
            # Tetangga dalam urutan berlawanan arah jarum jam mulai dari 9 o'clock
            neighbors = [gray[i - 1, j - 1], gray[i - 1, j], gray[i - 1, j + 1],
                         gray[i, j + 1], gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1]]
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            lbp_anticlockwise[i - 1, j - 1] = code
    return lbp_anticlockwise


def calculate_ncdblbp_clockwise(image):
    gray, rows, cols, ncdlbp_clockwise = pre_lbp(image)

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0            
            # Tetangga dalam urutan searah jarum jam mulai dari 3 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j + 1], gray[i, j + 1],
                         gray[i + 1, j + 1], gray[i + 1, j], gray[i + 1, j - 1],
                         gray[i, j - 1], gray[i - 1, j - 1]]
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            ncdlbp_clockwise[i, j] = code
    return ncdlbp_clockwise


def calculate_ncdblbp_anticlockwise(image):
    gray, rows, cols, ncdlbp_anticlockwise = pre_lbp(image)
    
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = gray[i, j]
            code = 0
            
            # Tetangga dalam urutan berlawanan arah jarum jam mulai dari 9 o'clock
            neighbors = [gray[i - 1, j], gray[i - 1, j - 1], gray[i, j - 1],
                         gray[i + 1, j - 1], gray[i + 1, j], gray[i + 1, j + 1],
                         gray[i, j + 1], gray[i - 1, j + 1]]
            
            # Hitung perbedaan antara piksel tetangga dan piksel pusat
            for k in range(8):
                if neighbors[k] >= center:
                    code |= (1 << k)
            
            ncdlbp_anticlockwise[i, j] = code    
    return ncdlbp_anticlockwise



def process_image(subfolder_path, file, directory, data):
    file_path = os.path.join(subfolder_path, file)
    image_path = file_path
    image = cv2.imread(image_path)
    
    temp_lbp = calculate_ncdblbp_anticlockwise(image)
    temp_lbp = temp_lbp.flatten()
    temp_lbp = temp_lbp.astype(object)
    temp_lbp = np.insert(temp_lbp, 0, directory)
    data.append(temp_lbp)


def save_dataframe_as_csv(data, out_dir, folder_name):
    df = pd.DataFrame(data)

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    df.to_csv(f'{out_dir}/{folder_name}.csv')

### `Config`

In [13]:
folder_names = ['dataset-resize-16', 'dataset-resize-32', 'dataset-resize-64', 'dataset-resize-128']
root_dir = './dataset-resize/'
out_dir = './csv-resize'

full_paths = [os.path.join(root_dir, folder) for folder in folder_names]

### `Main`

In [20]:
for i, path in enumerate(full_paths):
    data = []

    for root, directories, files in os.walk(path):
        folder_name = os.path.basename(root)

        for directory in directories:
            subfolder_path = os.path.join(root, directory)

            for file in os.listdir(subfolder_path):
                process_image(subfolder_path, file, directory, data)
                
    save_dataframe_as_csv(data, out_dir, folder_names[i])

In [21]:
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16375,16376,16377,16378,16379,16380,16381,16382,16383,16384
0,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
